<a href="https://colab.research.google.com/github/Brownwang0426/transformer-from-scratch/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 導入官方套件

In [27]:


import numpy as np
import math


import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, TensorDataset, Subset

import csv

import multiprocessing as mp
import os
import sys
import copy
import random
import gc
import time
from tqdm import tqdm
from collections import defaultdict

import itertools

import dill

import warnings
warnings.filterwarnings('ignore')

from datasets import load_dataset

import torch
from transformers import BertTokenizer, BertModel
import numpy as np

from IPython.display import display, clear_output

# 導入客製化套件

In [28]:
from model import *

# 確認有無讀取到 cuda

In [29]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    device_index = 0
    device = torch.device(f"cuda:{device_index}")
    print('using cuda...')
else:
    device = torch.device("cpu")
    print('using cpu...')

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

Device 0: NVIDIA GeForce RTX 4090
using cuda...


# 參數區域

In [30]:


max_length = 200

sequence_size =  max_length             
feature_size = 768          
num_layers = 3                      
num_heads = 4                
hidden_activation = 'tanh'
output_activation = 'tanh'
initializer = "xavier_normal"
optimizer = 'adam'
loss = 'mean_squared_error'
bias = False
drop_rate = 0.0
alpha = 0.0000001       

num_epochs = 100  
batch_size = 1

model_directory = f'model.pth'


In [31]:

response_length = 50

# 建立模型

In [32]:

# 建立模型
model = build_model(sequence_size,
                    feature_size,
                    num_layers,
                    num_heads,
                    hidden_activation,
                    output_activation,
                    initializer,
                    optimizer,
                    loss,
                    bias,
                    drop_rate,
                    alpha)

# 將模型放到 cuda
model = model.to(device)

# 讀取參數
model_dict = torch.load(model_directory)
model.load_state_dict(model_dict[f'model'])

<All keys matched successfully>

# 來跟這個小模型用英文聊聊天吧

In [34]:
# 你要問的句子
sentence = "where is taiwan located?"

In [35]:


# 初始化 BERT tokenizer 和 vectorizer
tokenizer  = BertTokenizer.from_pretrained('bert-base-uncased')
vectorizer = BertModel.from_pretrained('bert-base-uncased')

# 幫你的問題補上 [CSL] 以及 [SEP] ，讓機器可以知道問句的開始與結束
sentence = "[CLS] " + sentence + " [SEP] "

# 機器的回答
response = ''

# 開始遞迴
for i in range(response_length):

    # 將 QA tokenize
    tokenized_sentence = tokenizer(sentence, add_special_tokens=False, padding='max_length', max_length=max_length, truncation=True, return_tensors="pt")
    
    # 將 QA vectorize
    input_id           = tokenized_sentence['input_ids']
    attention_mask     = tokenized_sentence['attention_mask']
    with torch.no_grad(): 
        input_vector   = vectorizer(input_id).last_hidden_state * ( attention_mask.unsqueeze(2) ) 
    input_vector = input_vector.to(device)

    # 製作對應 QA 長度的 mask
    mask_2 = attention_mask[0].unsqueeze(1) * attention_mask[0].unsqueeze(0)
    mask_2 = mask_2.unsqueeze(0).unsqueeze(0)
    mask_2 = mask_2.to(device)
    mask_1 = (mask_2 -1) * 1e20
    mask_1 = mask_1.to(device)

    # 將你的問題向量丟到模型去吧
    model.eval()     
    output                 = model(input_vector, (mask_1, mask_2))

    # 依照機器人吐出的向量去找最可能的字是哪一個
    vocab_embeddings       = vectorizer.get_input_embeddings().weight .to(device)
    cos_sim                = F.cosine_similarity(output, vocab_embeddings, dim=1)
    most_similar_token_idx = torch.argmax(cos_sim).item()

    # 將機器人吐出的那個字拼接回去
    if most_similar_token_idx != 102:
        word = tokenizer.convert_ids_to_tokens(most_similar_token_idx)
        sentence += ' ' + word
        response += ' ' + word
        clear_output(wait=True)  # Clear the previous output
        print(response, flush=False)
        display()  # Display the updated output
    else:
        print("[END]")
        break



 the the . the the . . . ware . , , broken , madonna faded ##row losses ##ed orange orange quit consoles platinum musicians musicians
[END]
